In [1]:
from datetime import datetime, timedelta
import json
from pytz import timezone as tz
import requests
from functools import lru_cache

In [2]:
BASE_URL: str = "https://statsapi.mlb.com/api/v1"
CURR_SEASON: str = str(datetime.now().year)

def getData(endpoint: str, **kwargs) -> dict:
    queryParams: str = (
        f"&{'&'.join(f'{k}={v}' for k, v in kwargs.items())}" if kwargs else ""
    )
    url = f"{BASE_URL}/{endpoint}?sportId=1&season={CURR_SEASON}{queryParams}"
    return requests.get(url).json()


def printSample(endpoint: str, **kwargs) -> None:
    sampleTitle = kwargs.get(
        "sampleTitle", f"MLB {endpoint.capitalize()} Response Sample"
    )
    sampleSize = kwargs.get("size", None)
    print(f"\n{'='*len(sampleTitle)}\n{sampleTitle}\n{'='*len(sampleTitle)}\n")
    response = getData(endpoint, **kwargs).get(endpoint, [])
    sampleSize = kwargs.get(
        "sampleSize", len(response) if not sampleSize else sampleSize
    )
    data = response[0] if len(response) == 1 else response[0:sampleSize]
    print(json.dumps(data, indent=2))


In [3]:
@lru_cache(maxsize=1)
def _teamsIndex() -> dict:
    index = {}
    for team in getData("teams").get("teams", []):
        aliases = {
            team.get("name"),
            team.get("teamName"),
            team.get("abbreviation"),
            team.get("shortName"),
            team.get("locationName"),
        }
        teamId = team.get("id")
        for alias in aliases:
            if alias:
                index[alias.lower()] = teamId
    return index


def getTeamId(teamName: str) -> int | None:
    return _teamsIndex().get(teamName.lower())


In [16]:
@lru_cache(maxsize=1)
def _playersIndex() -> dict:
    players = (
        requests.get("https://statsapi.mlb.com/api/v1/sports/1/players")
        .json()
        .get("people", [])
    )
    index = {}
    for player in players:
        aliases = {
            player.get("firstLastName"),
            player.get("lastFirstName"),
        }
        playerId = player.get("id")
        for alias in aliases:
            if alias:
                index[alias.lower()] = playerId
    return index


def getPlayerId(playerName: str) -> int | None:
    return _playersIndex().get(playerName.lower())


In [17]:
def getLogoUrl(teamId: int | None = None, teamName: str | None = None, **kwargs) -> str:
    teamId = teamId if teamId else getTeamId(teamName) if teamName else None
    logoType = kwargs.get("logoType", "cap")
    logoShade = kwargs.get("logoShade", "light")
    return f"https://www.mlbstatic.com/team-logos/team-{logoType}-on-{logoShade}/{teamId}.svg"


def getHeadshotUrl(playerId: int | None = None, playerName: str | None = None) -> str:
    playerId = playerId if playerId else getPlayerId(playerName) if playerName else None
    return f"https://img.mlbstatic.com/mlb-photos/image/upload/d_people:generic:headshot:67:current.png/w_213,q_auto:best/v1/people/{playerId}/headshot/67/current"


In [18]:
def printTeamTotalSeasonWins(teamName: str, season: int = None) -> None:
    teamId = getTeamId(teamName)
    year = season or CURR_SEASON
    isCurrSeason = year == CURR_SEASON

    gameDates = getData(
        "schedule", teamId=teamId, startDate=f"{year}-01-01", endDate=f"{year}-12-31"
    ).get("dates", [])

    totalWins = 0
    for games in gameDates:
        for game in games.get("games", []):
            teams = game.get("teams", {})
            homeTeamResult = {
                "id": teams.get("home", {}).get("team", {}).get("id"),
                "isWinner": teams.get("home", {}).get("isWinner", False),
            }
            awayTeamResult = {
                "id": teams.get("away", {}).get("team", {}).get("id"),
                "isWinner": teams.get("away", {}).get("isWinner", False),
            }
            winningTeamId = (
                homeTeamResult["id"]
                if homeTeamResult["isWinner"]
                else awayTeamResult["id"] if awayTeamResult["isWinner"] else None
            )

            if winningTeamId == teamId:
                totalWins += 1